#### 生成 Title View需要的数据
暂定群体是all，直接使用titles_all.csv数据


In [1]:
import pandas as pd
import json
from tqdm import tqdm
import random
import math

titles_all_df = pd.read_csv('题目数据特征/titles_all.csv')
titleInfo_df = pd.read_csv("../dataset/Data_TitleInfo.csv")
st_df = pd.read_csv("人题数据/stu_title.csv")

In [51]:
subs = sorted(titleInfo_df['sub_knowledge'].unique())
kls = sorted(titleInfo_df['knowledge'].unique())

OutputArr = []
for kl in kls:
    dataobj = {"knowledge":kl}
    tmpsubs = []
    for sub in subs:
        if sub[:len(kl)] == kl:
            tmpsubs.append(sub)
    dataobj["sub_knowledges"]=tmpsubs
    List =  titleInfo_df[titleInfo_df['knowledge']==kl]['title_ID'].unique().tolist()
    dataobj["titleNum"] = len(List)
    # 提交次数和得分表现
    tmplist = []
    for title in List:
        titleObj = {}
        titleObj["t_id"] = title
        
        tdf = titles_all_df[titles_all_df['title_ID']==title]
        titleObj["belong"] = tdf["sub_knowledge"].tolist()[0]
        titleObj["t_mean_sbmnum"] = tdf['t_mean_sbmnum'].tolist()[0]
        titleObj["t_mean_score"] = tdf['t_mean_score'].tolist()[0]
        
        # 提交空间复杂度表现
        titleObj["t_mean_pfm_mem"] = tdf['t_mean_pfm_mem'].tolist()[0]
        titleObj["t_max_pfm_mem"] = tdf['t_max_pfm_mem'].tolist()[0]
        titleObj["t_min_pfm_mem"] = tdf['t_min_pfm_mem'].tolist()[0]
        # 提交时间复杂度表现
        titleObj["t_mean_pfm_tc"] = tdf['t_mean_pfm_tc'].tolist()[0]
        titleObj["t_max_pfm_tc"] = tdf['t_max_pfm_tc'].tolist()[0]
        titleObj["t_min_pfm_tc"] = tdf['t_min_pfm_tc'].tolist()[0]
        
        titdf = st_df[st_df['title_ID']==title]
        fulscore = titdf['score'].tolist()[0]
        titleObj["t_sbmNum"] = titdf.shape[0]
        titleObj["t_Fullscore"] = fulscore
        sc = titdf['st_score'].value_counts().to_dict()
        newobj = {}
        for key, value in sc.items():
            k = key * fulscore
            newobj[k] = value/titdf.shape[0]
        newobj = {k:newobj[k] for k in sorted(newobj)}
        
        obj ={"a":[], "b":[], "c":[]}
        v = 0
        for key, value in newobj.items():
            obj['a'].append(key)
            obj['b'].append(value)
            obj['c'].append(v)
            v = v+ value
        print(obj)
        titleObj["t_score_distribute"] = obj
        
        tmplist.append(titleObj)
    dataobj["titles"] = tmplist
    
    OutputArr.append(dataobj)
    

{'a': [0.0, 1.0, 2.0, 3.0], 'b': [0.0547945205479452, 0.03272450532724505, 0.0517503805175038, 0.860730593607306], 'c': [0, 0.0547945205479452, 0.08751902587519025, 0.13926940639269406]}
{'a': [0.0, 3.0], 'b': [0.13379737045630316, 0.8662026295436969], 'c': [0, 0.13379737045630316]}
{'a': [0.0, 1.0, 2.0, 3.0, 4.0], 'b': [0.06268882175226587, 0.00906344410876133, 0.0445619335347432, 0.013595166163141994, 0.8700906344410876], 'c': [0, 0.06268882175226587, 0.0717522658610272, 0.1163141993957704, 0.1299093655589124]}
{'a': [0.0, 1.0, 2.0, 3.0], 'b': [0.06681852695520121, 0.0425208807896735, 0.031131359149582385, 0.8595292331055429], 'c': [0, 0.06681852695520121, 0.10933940774487472, 0.1404707668944571]}
{'a': [0.0, 1.0, 2.0, 3.0], 'b': [0.006661732050333087, 0.011102886750555145, 0.1384159881569208, 0.843819393042191], 'c': [0, 0.006661732050333087, 0.017764618800888234, 0.15618060695780903]}
{'a': [0.0, 1.0, 2.0, 3.0], 'b': [0.09581749049429658, 0.019011406844106463, 0.023574144486692015,

In [46]:
json_str = json.dumps(OutputArr, ensure_ascii=False)
with open("TitleViewData.json", "w", encoding="utf-8") as file:
    file.write(json_str)

找各个指标的最大最小值帮助前端比例尺映射

In [33]:
min_pfm_tc = titles_all_df['t_min_pfm_tc'].min()
max_pfm_tc = titles_all_df['t_max_pfm_tc'].max()
print('时间复杂度范围',min_pfm_tc, max_pfm_tc)

min_pfm_mem = titles_all_df['t_min_pfm_mem'].min()
max_pfm_mem = titles_all_df['t_max_pfm_mem'].max()
print('空间复杂度范围',min_pfm_mem, max_pfm_mem)

min_mean_score = titles_all_df['t_mean_score'].min()
max_mean_score = titles_all_df['t_mean_score'].max()
print('平均分范围',min_mean_score, max_mean_score)
min_mean_sbmnum = titles_all_df['t_mean_sbmnum'].min()
max_mean_sbmnum = titles_all_df['t_mean_sbmnum'].max()
print('平均提交次数范围',min_mean_sbmnum, max_mean_sbmnum)

时间复杂度范围 1 400
空间复杂度范围 128 65536
平均分范围 13.601599165652702 49.81823577416723
平均提交次数范围 2.4038004750593824 8.848640705363703
